In [5]:
#imports
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json
from datetime import datetime
import PIL.Image
import sys
from PIL import Image
sys.modules['Image'] = Image
import io
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [14]:
#hyperparameters and constants definition
# to run it on another machine, just put the notebook file aside the MaskDataset folder

my_name = 'hw1_final'

#paths
base_folder = ''
data_folder =  os.path.join(base_folder, 'MaskDataset')
train_folder = os.path.join(data_folder, 'training')
test_folder = os.path.join(data_folder, 'test')
json_train_path = os.path.join(data_folder, 'train_gt.json')
exps_dir = os.path.join(base_folder, 'exp_2')
results_folder = os.path.join(base_folder, 'results')
hyp_json_path = os.path.join(base_folder, 'hyp_config.json')

#if true, it will load the hyperparameters from json
load_hyp_json = False

if load_hyp_json:
    with open(hyp_json_path)as json_hyp:
        hyp_config = json.load(json_hyp)
# otherwise you can set the parameters here
else:
    hyp_config = {
        "SEED": 1234,
        "bs": 16,
        "train_split": 0.75,
        "img_w": 612,
        "img_h": 612,
        "apply_data_augmentation": False,
        "lr": 1e-5,
        "dropout_factor": 0.3, 
        "create_cm": False
    }
num_classes = 3
n_epochs = 50

#random seed
SEED = hyp_config["SEED"]

#batch size
bs = hyp_config["bs"] 

#percentage of images used for training
train_split = hyp_config["train_split"]

#img shape
img_w = hyp_config["img_w"]
img_h = hyp_config["img_h"]

#if true, data augmentation will be applied
apply_data_augmentation = hyp_config["apply_data_augmentation"]

#learning rate
lr = hyp_config["lr"]

#dropout
dropout_factor = hyp_config["dropout_factor"]

In [15]:
#ImageDataGenerator creation
tf.random.set_seed(SEED)

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range = 10,
                                        width_shift_range = 21,
                                        height_shift_range = 21,
                                        zoom_range = 0.2,
                                        horizontal_flip = True,
                                        vertical_flip = False,
                                        fill_mode = 'constant',
                                        cval = 0,
                                        rescale = 1./255)
else:
    train_data_gen = ImageDataGenerator(rescale = 1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [16]:
# load the dataframe with the labels of the training set
with open(json_train_path) as json_train:
    dict_train = json.load(json_train)
    
dataframe = pd.DataFrame(dict_train.items())
dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe["class"] = dataframe["class"].astype(str)

#split the dataset into training and validation sets
train_df = dataframe.sample(frac = train_split, random_state = SEED)
valid_df = dataframe.drop(train_df.index)

train_gen = train_data_gen.flow_from_dataframe(train_df,
                                               train_folder,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

valid_gen = valid_data_gen.flow_from_dataframe(valid_df,
                                               train_folder,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

train_gen.class_indices


Found 4210 validated image filenames belonging to 3 classes.
Found 1404 validated image filenames belonging to 3 classes.


{'0': 0, '1': 1, '2': 2}

In [17]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()

In [18]:
start_f = 12
depth = 6

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                 kernel_size=(5, 5),
                                 strides=(2, 2),
                                 padding='same',
                                 input_shape=[img_h, img_w, 3]))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
start_f *= 2

for i in range(depth):

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=[None]))
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=[None]))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    if (i > 2): model.add(tf.keras.layers.Dropout(dropout_factor))
    start_f *= 2

    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dropout(dropout_factor))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dropout(dropout_factor))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))


# Visualize created model as a table
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 306, 306, 12)      912       
_________________________________________________________________
re_lu_13 (ReLU)              (None, 306, 306, 12)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 153, 153, 12)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 153, 153, 24)      2616      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 153, 153, 24)      5208      
_________________________________________________________________
re_lu_14 (ReLU)              (None, 153, 153, 24)      0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 76, 76, 24)       

In [19]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [21]:
#creates folders to save logs during training
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = my_name
exp_name = model_name + '_' + str(now)

exp_dir = os.path.join(exps_dir, exp_name)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
#save configuration and hyperparameters
config = model.to_json()
json_model_path = os.path.join(exp_dir, 'model.json')
with open(json_model_path, 'w') as f:
    json.dump(config, f)
    
with open(os.path.join(exp_dir, 'hyp_config.json'), 'w') as f:
    json.dump(hyp_config, f)
    
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=False)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)


# saves also a log in csv format for fast plotting
csv_path = os.path.join(exp_dir, 'log.csv')
log_callback = tf.keras.callbacks.CSVLogger(csv_path, append=True, separator=';')
callbacks.append(log_callback)


# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    callbacks.append(es_callback)

In [22]:
history = model.fit(x=train_dataset,
          epochs=n_epochs,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 264 steps, validate for 88 steps
Epoch 1/50
 26/264 [=>............................] - ETA: 1:16 - loss: 1.0967 - accuracy: 0.3675WARNING:tensorflow:From C:\Users\Michele\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: exp_2\hw1_final_Nov22_02-42-53\ckpts\cp_01.ckpt\assets
 26/264 [=>............................] - ETA: 2:17 - loss: 1.0967 - accuracy: 0.3675

KeyboardInterrupt: 

In [18]:
def create_csv(results, results_dir='./'):
    csv_fname = 'results_'
    csv_fname += exp_name + '___' + datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:
        f.write('Id,Category\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

images = [f for f in os.listdir(test_folder)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = test_data_gen.flow_from_dataframe(images,
                                              test_folder,
                                              batch_size=bs,
                                              target_size=(img_h, img_w),
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results, results_folder)

Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
29/29 [==============================] - 36s 1s/step
